##### Part 1
Load the MNIST data, and split it into a training set, a validation set, and a test set (e.g., use 40,000 instances for training, 10,000 for validation, and 10,000 for testing, use random_state = 2017). Then train a Random Forest classifier, an Extra-Trees classifier, and an SVM. Next, try to combine them into an ensemble that outperforms them all on the validation set, using a soft and hard voting classifier. Once you have found one, try it on the test set. How much better does it perform compared to the individual classifiers?

## Loading the MNIS data and split it into traning set, a validation set, and a test set

In [45]:
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')

Due to large data SVM algorithm is not working while performing the voting classifer. Creating the sample data for resolve the problem 

In [46]:
X=mnist['data']
y=mnist['target']


In [47]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test   = train_test_split(X, y, train_size=1/10, random_state=2017,stratify=y)
X_train, X_test, y_train, y_test   = train_test_split(X_train, y_train, test_size=1/7, random_state=2017,stratify=y_train)
X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=1/6, random_state=2017,stratify=y_train)

C:\Users\Rushabh Shah\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


## Train a Random Forest classifier, an Extra-Trees classifier, and an SVM

In [48]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

ext_clf = ExtraTreesClassifier(random_state=2017)
rnd_clf = RandomForestClassifier(random_state=2017)
svm_clf = SVC(kernel='linear',random_state=2017)

## Hard voting classifier

In [49]:
voting_clf = VotingClassifier(estimators=[('et', ext_clf), ('rf', rnd_clf), ('svc', svm_clf)], voting='hard')
voting_clf.fit(X_train, y_train)


VotingClassifier(estimators=[('et', ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_...,
  max_iter=-1, probability=False, random_state=2017, shrinking=True,
  tol=0.001, verbose=False))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [50]:
from sklearn.metrics import accuracy_score

for clf in (ext_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    print(clf.__class__.__name__, accuracy_score(y_val, y_pred))

ExtraTreesClassifier 0.891
RandomForestClassifier 0.883
SVC 0.9
VotingClassifier 0.91


## Soft voting classifier

In [51]:
ext_clf = ExtraTreesClassifier(random_state=2017)
rnd_clf = RandomForestClassifier(random_state=2017)
svm_clf = SVC(kernel='linear',probability=True,random_state=2017)



voting_clf = VotingClassifier(estimators=[('et', ext_clf), ('rf', rnd_clf), ('svc', svm_clf)], voting='soft')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('et', ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_...',
  max_iter=-1, probability=True, random_state=2017, shrinking=True,
  tol=0.001, verbose=False))],
         flatten_transform=None, n_jobs=1, voting='soft', weights=None)

In [52]:
from sklearn.metrics import accuracy_score

for clf in (ext_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    print(clf.__class__.__name__, accuracy_score(y_val, y_pred))

ExtraTreesClassifier 0.891
RandomForestClassifier 0.883
SVC 0.9
VotingClassifier 0.928


## Observation:

Hard voting classifer gives accuracy=0.91 while Soft voting classifer gives accuracy=0.928. So running the Soft voting classifer on Test data and find the accuracy of the model.

## Soft voting classifier on Test Data

In [53]:
ext_clf = ExtraTreesClassifier(random_state=2017)
rnd_clf = RandomForestClassifier(random_state=2017)
svm_clf = SVC(kernel='linear',probability=True,random_state=2017)



voting_clf = VotingClassifier(estimators=[('et', ext_clf), ('rf', rnd_clf), ('svc', svm_clf)], voting='soft')
voting_clf.fit(X_train, y_train)


for clf in (ext_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

ExtraTreesClassifier 0.871
RandomForestClassifier 0.879
SVC 0.892
VotingClassifier 0.917


## Observation:

Soft voting classifer outperforms all othrer individual classifiers with accuracy score =0.917. Among the all three individual classifiers, SVC algorithm has best accuracy = 0.892

##### Part 2
On MNIST dataset with above split, train a Random Forest classifier on the dataset and time how long it takes, then evaluate the resulting model on the test set. Next, use PCA to reduce the dataset’s dimensionality, with an explained variance ratio of 95%. Train a new Random Forest classifier on the reduced dataset and see how long it takes. Was training much faster? Next evaluate the classifier on the test set: how does it compare to the previous classifier?

## Loading the MNIS data and split it into traning set, a validation set, and a test set

In [54]:
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')
X=mnist['data']
y=mnist['target']

In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test   = train_test_split(X, y, test_size=1/7, random_state=2017,stratify=y)

## Train a Random Forest classifier on the dataset

In [56]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(random_state=2017)

## Measuring the time and accuracy of the model before reducing the dataset’s dimensionality

In [57]:
import time

t0 = time.time()
rnd_clf.fit(X_train, y_train)
t1 = time.time()

In [58]:
print("Training took {:.2f}s".format(t1 - t0))

Training took 4.41s


In [59]:
from sklearn.metrics import accuracy_score

y_pred = rnd_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.94940000000000002

## Running PCA to reduce the dataset’s dimensionality, with an explained variance ratio of 95%

In [60]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)
X_train_reduced = pca.fit_transform(X_train)

## Measuring the time and accuracy of the model after reducing the dataset’s dimensionality

In [61]:
rnd_clf2 = RandomForestClassifier(random_state=42)
t0 = time.time()
rnd_clf2.fit(X_train_reduced, y_train)
t1 = time.time()

In [62]:

print("Training took {:.2f}s".format(t1 - t0))

Training took 10.60s


In [63]:
X_test_reduced = pca.transform(X_test)

y_pred = rnd_clf2.predict(X_test_reduced)
accuracy_score(y_test, y_pred)

0.90510000000000002

## Observations:

It should be expected that dimensionality reduction does lead to faster training time but here is not the case.It acually took longer time(=10.60s) than time(=4.41s) without performing dimensionality reduction. It depends on the dataset, the model and the training algorithm

Accuracy: It is common for performance to drop slightly when reducing dimensionality, because we do lose some informatioe during the process. Here, the performance drop is severe.Model accuracy score is 0.9494 before dimenstion reduction and Model accuracy score is 0.9051 after dimenstion reduction.